# Test RLS on storage table

In [ ]:
from laminhub_rest.core.account import delete_account
from laminhub_rest.connector import connect_hub, connect_hub_with_auth
import pytest
from laminhub_rest.utils._test import (
    create_test_auth,
    create_test_account,
    create_test_storage,
)
from laminhub_rest.core.storage._crud import (
    sb_insert_storage,
    sb_select_storage,
    sb_udpate_storage,
    sb_delete_storage,
)
from uuid import uuid4
import string, secrets
from laminhub_rest._clean_ci import clean_ci

In [ ]:
def base62(n_char: int) -> str:
    """Like nanoid without hyphen and underscore."""
    alphabet = string.digits + string.ascii_letters.swapcase()
    id = "".join(secrets.choice(alphabet) for i in range(n_char))
    return id

## Parametrize 

In [ ]:
hub = connect_hub()

# Create account 1
auth_1 = create_test_auth()
access_token_1 = auth_1["access_token"]
account_1 = create_test_account(handle=auth_1["handle"], access_token=access_token_1)
account_id_1 = account_1["id"]
account_handle_1 = account_1["handle"]
account_hub_1 = connect_hub_with_auth(access_token=access_token_1)

# Create account 2
auth_2 = create_test_auth()
access_token_2 = auth_2["access_token"]
account_2 = create_test_account(handle=auth_2["handle"], access_token=access_token_2)
account_id_2 = account_2["id"]
account_handle_2 = account_2["handle"]
account_hub_2 = connect_hub_with_auth(access_token=access_token_2)

## Enable authenticated accounts to create storage

In [ ]:
storage_id = uuid4().hex
storage_lnid = (base62(8),)

With an anonymous Supabase client

In [ ]:
with pytest.raises(Exception) as error:
    storage = sb_insert_storage(
        storage_fields={
            "id": storage_id,
            "lnid": storage_lnid,
            "created_by": account_id_1,
            "root": f"lamin.ci.storage.{base62(6)}",
            "region": "us-east-1",
            "type": "s3",
        },
        supabase_client=hub,
    )

assert "new row violates row-level security policy" in error.value.message

Using another account authentication

In [ ]:
with pytest.raises(Exception) as error:
    storage = sb_insert_storage(
        storage_fields={
            "id": storage_id,
            "lnid": storage_lnid,
            "created_by": account_id_1,
            "root": f"lamin.ci.storage.{base62(6)}",
            "region": "us-east-1",
            "type": "s3",
        },
        supabase_client=account_hub_2,
    )

assert "new row violates row-level security policy" in error.value.message

Using corresponding account authentication

In [ ]:
storage = sb_insert_storage(
    storage_fields={
        "id": storage_id,
        "lnid": storage_lnid,
        "created_by": account_id_1,
        "root": f"lamin.ci.storage.{base62(6)}",
        "region": "us-east-1",
        "type": "s3",
    },
    supabase_client=account_hub_1,
)

assert storage is not None

## Enable everyone to select a storage

With an anonymous Supabase client

In [ ]:
storage = sb_select_storage(id=storage_id, supabase_client=hub)

assert storage is not None

## Enable owner to update their storage

With an anonymous Supabase client

In [ ]:
storage = sb_udpate_storage(
    id=storage_id,
    storage_fields={
        "region": "us-east-2",
    },
    supabase_client=hub,
)

assert storage is None
assert sb_select_storage(id=storage_id, supabase_client=hub)["region"] == "us-east-1"

Using another account authentication

In [ ]:
storage = sb_udpate_storage(
    id=storage_id,
    storage_fields={
        "region": "us-east-2",
    },
    supabase_client=account_hub_2,
)

assert storage is None
assert sb_select_storage(id=storage_id, supabase_client=hub)["region"] == "us-east-1"

Using corresponding account authentication

In [ ]:
storage = sb_udpate_storage(
    id=storage_id,
    storage_fields={
        "region": "us-east-2",
    },
    supabase_client=account_hub_1,
)

assert storage is not None
assert sb_select_storage(id=storage_id, supabase_client=hub)["region"] == "us-east-2"

## Enable owner to delete their storage

With an anonymous Supabase client

In [ ]:
storage = sb_delete_storage(id=storage_id, supabase_client=hub)

assert storage is None
assert sb_select_storage(id=storage_id, supabase_client=hub) is not None

Using another account authentication

In [ ]:
storage = sb_delete_storage(id=storage_id, supabase_client=account_hub_2)

assert storage is None
assert sb_select_storage(id=storage_id, supabase_client=account_hub_2) is not None

Using corresponding account authentication

In [ ]:
storage = sb_delete_storage(id=storage_id, supabase_client=account_hub_1)

assert storage is not None
assert sb_select_storage(id=storage_id, supabase_client=account_hub_1) is None

## Clean up test assets

In [ ]:
delete_account(account_handle_1, access_token_1)
delete_account(account_handle_2, access_token_2)

In [ ]:
clean_ci()